### Load Libraries

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import accuracy_score
import sys
from keras import regularizers
import pandas

Using TensorFlow backend.


### Load Raw Training Data

In [2]:
inpData = []
labels = []

'''
with open('cdata.csv') as csv_file:
    lines = csv.reader(csv_file, delimiter=',')
    for row in lines:
        labels.append(row[0])
        inpData.append(float(row[1]))
'''
inpDataArray = np.loadtxt("usageData.csv",delimiter = ',') 

    

### Set the Target Labels

In [3]:
minThreshold = 30
maxThreshold = 70
target = []
cpuUsage = []
historyParam = 50
for i in range(0,inpDataArray.shape[1]):
    inpData = inpDataArray[:,i]
    index = historyParam
    while index < len(inpData):
        currRec = inpData[index-historyParam:index]
        if inpData[index] > maxThreshold:# Scale Up
            target.append(1)
        elif inpData[index] < minThreshold: # Scale Down
            target.append(-1)
        else:
            target.append(0)  #Remain same
        cpuUsage.append(currRec)
        index += 1

In [4]:
cpuUsageData = np.expand_dims(np.array(cpuUsage,dtype = 'float'),axis = 2)
cpuScaling = np.array(target,dtype = 'float')
print(cpuScaling.shape)
print(cpuUsageData.shape)

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)

(47072,)
(47072, 50, 1)


### Preprocessing Input Data

In [5]:
'''cpuUsage = []
cpuScaling = []
historyParam = 50
index = historyParam

while index<len(inpData):
    currRec = [[inpData[i]] for i in range(index-historyParam,index)]
    cpuUsage.append(currRec)
    index+=1'''
    

'cpuUsage = []\ncpuScaling = []\nhistoryParam = 50\nindex = historyParam\n\nwhile index<len(inpData):\n    currRec = [[inpData[i]] for i in range(index-historyParam,index)]\n    cpuUsage.append(currRec)\n    index+=1'

In [6]:
'''cpuScaling = target[historyParam-1:len(target)]


cpuUsageData = np.array(cpuUsage,dtype = 'float')
cpuScaling = np.array(cpuScaling,dtype = 'float')

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)'''

"cpuScaling = target[historyParam-1:len(target)]\n\n\ncpuUsageData = np.array(cpuUsage,dtype = 'float')\ncpuScaling = np.array(cpuScaling,dtype = 'float')\n\ncpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)"

In [7]:
#len(inpData)
#X_train, X_test, y_train, y_test = train_test_split(cpuUsageData , cpuScalingCategorical, test_size=0.01, random_state=42)
'''newTrainX = cpuUsageData
newTrainY = cpuScalingCategorical
for i in range(1,len(cpuUsageData)):
    scalingVal = np.argmax(cpuScalingCategorical[i])
    if scalingVal == 0:
        for j in range(0,7):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)
        
    if scalingVal == 1 :
        for j in range(0,10):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)'''


X_train, X_test, y_train, y_test = train_test_split(cpuUsageData , cpuScalingCategorical, test_size=0.10, random_state=3)

In [8]:
cpuScaling.shape
#np.shape(cpuUsage)

(47072,)

In [9]:
import pandas
testrec = 5
print(cpuScalingCategorical[testrec])
print(cpuScaling[testrec])
print(set(cpuScaling))
myseries=pandas.Series(cpuScaling)
print(myseries.value_counts())

[1. 0. 0.]
0.0
{0.0, 1.0, -1.0}
 0.0    20761
-1.0    13176
 1.0    13135
dtype: int64


In [10]:
usageInputLayer = keras.layers.Input(shape=(historyParam,1))
flattened_layer = keras.layers.Flatten()(usageInputLayer)
'''fullyConnected_0 = keras.layers.Dense(125, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(flattened_layer)'''
fullyConnected_1 = keras.layers.Dense(50, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(flattened_layer)
#drop1 = keras.layers.Dropout(0.25)(fullyConnected_1)
fullyConnected_2 = keras.layers.Dense(50, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_1)
#drop2 = keras.layers.Dropout(0.25)(fullyConnected_2)
fullyConnected_3 = keras.layers.Dense(50, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_2)
fullyConnected_4 = keras.layers.Dense(25, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_3)
fullyConnected_5 = keras.layers.Dense(12, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_4)
fullyConnected_6 = keras.layers.Dense(6, activation = 'relu',
                kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.1))(fullyConnected_5)
softmax_output = keras.layers.Dense(3,activation='softmax',kernel_initializer='he_normal')(fullyConnected_6)
predictionModel = keras.models.Model(inputs=usageInputLayer,outputs=softmax_output)
predictionModel.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 1)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
dens

In [11]:
def customLoss(rewardInputLayer):
    def loss(y_true,y_pred):
        tmp_pred = keras.layers.Lambda(lambda x: keras.backend.clip(x,0.05,0.95))(y_pred)
        tmp_loss = keras.losses.categorical_crossentropy(y_true, y_pred)
        policy_loss=keras.layers.Multiply()([tmp_loss,rewardInputLayer])
        #policy_loss = tf.reduce_sum(rewardInputLayer,axis =-1)
        return policy_loss
    return loss

In [12]:
rewardInputLayer = keras.layers.Input(shape=(1,),name='rewardInputLayer')
TrainingModel = keras.models.Model(inputs=[usageInputLayer,rewardInputLayer],outputs=softmax_output)

RMSPropOptimizer = keras.optimizers.RMSprop(lr=0.001)
NadamOptimizer = keras.optimizers.Nadam(lr=0.0001)
AdamOptimizer = keras.optimizers.Adam(lr=0.0001)
TrainingModel.compile(optimizer=AdamOptimizer,loss=customLoss(rewardInputLayer))

In [13]:
TrainingModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 1)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_5 (Dense)              (None, 12)                312       
__________

In [14]:
def generateReward(usage,action,minThreshold,maxThreshold):
    if usage[49] > maxThreshold :
        if action == 0:
            return 1
        elif action == -1:
            return 2
    elif usage[49] < minThreshold: 
        if action == 0:
            return 1
        elif action == 1:
            return 2
    else:
        if action != 0:
            return 1
    return -1


def generateReward2(requiredAction,action):
    predDict = {0:0,1:1,2:-1}
    actualActions = predDict[np.argmax(requiredAction)]
    if(actualActions == action):
        reward = 1
    else:
        reward = -1
    return reward
    
def simulateSituation(pModel,tModel,usageArray,minThreshold,maxThreshold):
    predDict = {0:0,1:1,2:-1}
    #modelInput = np.expand_dims(usageArray,axis=0)
    res = pModel.predict(usageArray)
    #mapper = lambda x: predDict[x]
    actionPredictedOneHot = to_categorical(np.argmax(res,axis = 1),num_classes = 3)
    actionPredicted = np.array([predDict[np.argmax(x)] for x in res])
    rewardsList = []
    #print(actionPredicted," \n",usageArray[0])
    print(len(usageArray))
    
    for i in range(0,len(usageArray)):
        #actionReward = generateReward(y_train[i],actionPredicted[i],minThreshold,maxThreshold,)
        actionReward = generateReward2(y_train[i],actionPredicted[i])
        if actionReward == None:
            print(actionPredicted[i]," \n",usageArray[i]," ", minThreshold," ",maxThreshold)
            sys.exit()
            
        rewardsList.append(actionReward)
        
    rewardsArray = np.array(rewardsList)
    
    actionSeries = pandas.Series(actionPredicted)
    print(actionSeries.value_counts())
    rewardSeries = pandas.Series(rewardsList)
    print(rewardSeries.value_counts())
    #print(rewardsArray.shape," ",rewardsArray[0])
    #if os.path.isfile("LSTMRLBestWeights.hdf5"):
    #    print("Loading Previous weights!")
    #    tModel.load_weights("LSTMRLBestWeights.hdf5")
    history = tModel.fit(x = [usageArray,rewardsArray], y = actionPredictedOneHot,epochs = 1,batch_size = 1000)
    #print(usageArray[0]," ", actionPredicted," \n", rewardsArray)
    return pModel,tModel,res
    

In [16]:
#filepath="LSTMRLBestWeights.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

epochs = 100
for i in range(epochs):
    _,_,modelOutputOneHot = simulateSituation(predictionModel,TrainingModel,X_train,30,70)
    #simulateSituation(predictionModel,TrainingModel,X_train,30,70)
    res = predictionModel.predict(X_test)
    print(accuracy_score(np.argmax(modelOutputOneHot,axis = 1),np.argmax(y_train,axis = 1)))
    print(accuracy_score(np.argmax(res,axis = 1),np.argmax(y_test,axis = 1)))

42364
1    42364
dtype: int64
-1    30562
 1    11802
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 13.0690
0.2785855915399868
0.2831350892098556
42364
1    42364
dtype: int64
-1    30562
 1    11802
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 12.7114
0.2785855915399868
0.2831350892098556
42364
1    42364
dtype: int64
-1    30562
 1    11802
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 12.3617
0.2785855915399868
0.2831350892098556
42364
1    42364
dtype: int64
-1    30562
 1    11802
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 12.0197
0.2785855915399868
0.2831350892098556
42364
1    42364
dtype: int64
-1    30562
 1    11802
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 11.6854
0.2785855915399868
0.2831350892098556
42364
1    42364
dtype: int64
-1    30562
 1    11

42364/42364 [==============================] - 0s 6us/step - loss: 4.2228
0.8602587102256632
0.87107051826678
42364
 0    19742
 1    11693
-1    10929
dtype: int64
 1    36752
-1     5612
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 4.1034
0.8675290340855444
0.8787170773152082
42364
 0    20171
 1    11639
-1    10554
dtype: int64
 1    36941
-1     5423
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 3.9833
0.8719903691813804
0.8799915038232795
42364
 0    20020
 1    11397
-1    10947
dtype: int64
 1    37081
-1     5283
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 3.8637
0.8752950618449628
0.8840271877655055
42364
 0    20165
 1    11324
-1    10875
dtype: int64
 1    37217
-1     5147
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 3.7485
0.878505334718157
0.8853016142735769
42364
 0    19781
-1    11412
 1   

0.8997261826078746
0.9054800339847069
42364
 0    21257
 1    10615
-1    10492
dtype: int64
 1    38132
-1     4232
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 1.3085
0.9001038617694269
0.9039932030586236
42364
 0    21423
 1    10619
-1    10322
dtype: int64
 1    38089
-1     4275
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 8us/step - loss: 1.2713
0.8990888490227552
0.905267629566695
42364
 0    21159
 1    10610
-1    10595
dtype: int64
 1    38142
-1     4222
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 8us/step - loss: 1.2404
0.900339911245397
0.9059048428207307
42364
 0    21136
-1    10623
 1    10605
dtype: int64
 1    38149
-1     4215
dtype: int64
Epoch 1/1
42364/42364 [==============================] - 0s 6us/step - loss: 1.2084
0.9005051458785761
0.9067544604927783
42364
 0    20916
-1    10846
 1    10602
dtype: int64
 1    38210
-1     4154
dtype: int64
Epoch 1/1
42364/

In [17]:
predictionModel.save("3OutputRLV4BestWeights.hdf5")

In [ ]:
if os.path.isfile("3OutputRLBestWeights.hdf5"):
    print("Loading Previous weights!")
    TrainingModel.load_weights("3OutputRLBestWeights.hdf5")
res = predictionModel.predict(X_train)

In [ ]:
accuracy_score(np.argmax(res,axis = 1),np.argmax(y_train,axis = 1))

In [ ]:
print(to_categorical(np.argmax(y_train,axis = 1),num_classes = 3)[:10,:])
print(y_train[:10,:])

In [ ]:
np.argmax([[0,0,1],[1,0,0]],axis = 1)

In [ ]:
temp1 = np.array([[]])

In [ ]:
res = predictionModel.predict(X_train)

In [ ]:
res.shape

In [ ]:
predDict = {0:0,1:1,2:-1}
actionPredictedOneHot = to_categorical(np.argmax(res,axis = 1),num_classes = 3)
actionPredicted = np.array([predDict[np.argmax(x)] for x in res])
rewardList = []
for i in range(0,len(y_train)):
        #actionReward = generateReward(y_train[i],actionPredicted[i],minThreshold,maxThreshold,)
        actionReward = generateReward2(y_train[i],actionPredicted[i])
        rewardList.append(actionReward)
actionSeries = pandas.Series(actionPredicted)
print(actionSeries.value_counts())
rewardSeries = pandas.Series(rewardList)
print(rewardSeries.value_counts())

In [ ]:
resSeries = pandas.Series([predDict[np.argmax(x)] for x in y_train])
print(resSeries.value_counts())

In [ ]:
filepath="LSTMBestWeights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
if os.path.isfile("LSTMBestWeights.hdf5"):
    print("Loading Previous weights!")
    model.load_weights("LSTMBestWeights.hdf5")

In [ ]:
history = model.fit(X_train, y_train, epochs = 15,validation_data = [X_test, y_test], callbacks = callbacks_list,verbose=0)

In [ ]:
X_test.shape
np.expand_dims(X_test[0],axis = 0).shape

In [ ]:
res = model.predict(np.expand_dims(X_test[0],axis=0))

In [ ]:
print(res)
print(np.argmax(res[0]))

In [ ]:
np.argmax([0,1,0],axis = 0)

In [ ]:
np.argmax(res,axis = 1)

In [ ]:
print(inpData[0+1])

In [ ]:
cpuUsageData
cpuScalingCategorical
newTrainX = cpuUsageData
newTrainY = cpuScalingCategorical
for i in range(1,len(cpuUsageData)):
    scalingVal = np.argmax(cpuScalingCategorical[i])
    if scalingVal == 0:
        for j in range(0,7):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)
        
    if scalingVal == 1 :
        for j in range(0,10):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)
#newTrainX = [cpuUsageData[x] for x in range(1,len(cpuUsageData)] if np.argmax(cpuScalingCategorical[x])==0 ]

res2Series = pandas.Series([predDict[np.argmax(x)] for x in newTrainY])
print(res2Series.value_counts())